# 05 自动求梯度(二)

In [1]:
import torch

## 求梯度

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward1>)


In [5]:
out.backward()

In [6]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


## 反向传播，梯度累加

In [7]:
out2 = x.sum()
print(out2)

tensor(4., grad_fn=<SumBackward0>)


In [8]:
out2.backward()
print(x.grad)

tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])


## 通常反向传播前，梯度清零

In [9]:
out3 = x.sum()
print(out3)

tensor(4., grad_fn=<SumBackward0>)


In [10]:
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


不允许张量对张量求导，只允许标量对张量求导，求导结果为和自变量同形的张量

In [11]:
x1 = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y1 = 2 * x1
z1 = y1.view(2, 2)
print(x1)
print(y1)
print(z1)
# 这里y1不是一个标量

tensor([1., 2., 3., 4.], requires_grad=True)
tensor([2., 4., 6., 8.], grad_fn=<MulBackward0>)
tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


In [12]:
v = torch.tensor([[1.0, 1.0], [0.01, 0.001]], dtype=torch.float)
z1.backward(v)
print(x1.grad)

tensor([2.0000, 2.0000, 0.0200, 0.0020])


## 梯度追踪

In [13]:
x2 = torch.tensor(1.0, requires_grad=True)
y2_1 = x2 ** 2
with torch.no_grad():
    y2_2 = x2 ** 3
y2_3 = y2_1 + y2_2

In [14]:
print(x2)
print(x2.requires_grad)
print(y2_1)
print(y2_1.requires_grad)
print(y2_2)
print(y2_2.requires_grad)
print(y2_3)
print(y2_3.requires_grad)

tensor(1., requires_grad=True)
True
tensor(1., grad_fn=<PowBackward0>)
True
tensor(1.)
False
tensor(2., grad_fn=<AddBackward0>)
True


In [15]:
y2_3.backward()
print(x2.grad)

tensor(2.)


## 对tensor.data进行操作

修改tensor数值，但不会被autograd记录

In [16]:
x3 = torch.ones(1, requires_grad=True)

In [17]:
print(x3)
print(x3.data)
print(x3.data.requires_grad)
# x3不在计算图之内

tensor([1.], requires_grad=True)
tensor([1.])
False


In [18]:
y3 = 2 * x3
x3.data *= 100
# 只改变了值，没改变计算图
print(x3)
print(x3.data)

tensor([100.], requires_grad=True)
tensor([100.])


In [19]:
y3.backward()
print(x3)
print(x3.grad)

tensor([100.], requires_grad=True)
tensor([2.])
